In [1]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [2]:
#read in incidence csv from github

mortality_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Race_Mortality_Cancer_DB.csv")                         
#get rid of commas in columns
mortality_df = mortality_df.replace(',','', regex=True)
mortality_df.head()     

,average_annual_count,recent_trend,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,five_year_trend_rates,county_state,...,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,median_household_income_2015,median_household_income_2016,median_household_income_2017,median_household_income_2018,median_household_income_2019
0,181,stable,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,213.2,213.2,Walker County AL,...,64931,64533,63895,63669,63521,37305.0,39511.0,41925.0,45434.0,45991.0
1,180,stable,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,213.9,213.9,Walker County AL,...,64931,64533,63895,63669,63521,37305.0,39511.0,41925.0,45434.0,45991.0
2,10,stable,Alabama,All Cancer Sites,Black (includes Hispanic),All Sexes,All Ages,281.1,281.1,Walker County AL,...,64931,64533,63895,63669,63521,37305.0,39511.0,41925.0,45434.0,45991.0
3,32,stable,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,211.2,211.2,Crenshaw County AL,...,13850,13916,13858,13825,13772,36759.0,37374.0,38419.0,39057.0,43309.0
4,32,stable,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,214.0,214.0,Crenshaw County AL,...,13850,13916,13858,13825,13772,36759.0,37374.0,38419.0,39057.0,43309.0


In [3]:
#check dtypes
mortality_df.dtypes

average_annual_count                     int64
recent_trend                            object
primary_state_name                      object
cancer                                  object
race_ethnicity                          object
sex                                     object
age                                     object
age_adjusted_rate                       object
five_year_trend_rates                   object
county_state                            object
index_of_medical_underservice_score    float64
popestimate2015                          int64
popestimate2016                          int64
popestimate2017                          int64
popestimate2018                          int64
popestimate2019                          int64
median_household_income_2015           float64
median_household_income_2016           float64
median_household_income_2017           float64
median_household_income_2018           float64
median_household_income_2019           float64
dtype: object

In [4]:
#get count of rows
mortality_df.count()

average_annual_count                   7341
recent_trend                           7341
primary_state_name                     7341
cancer                                 7341
race_ethnicity                         7341
sex                                    7341
age                                    7341
age_adjusted_rate                      7341
five_year_trend_rates                  7341
county_state                           7341
index_of_medical_underservice_score    6439
popestimate2015                        7341
popestimate2016                        7341
popestimate2017                        7341
popestimate2018                        7341
popestimate2019                        7341
median_household_income_2015           7328
median_household_income_2016           7328
median_household_income_2017           7328
median_household_income_2018           7328
median_household_income_2019           7328
dtype: int64

In [5]:
#get count of values for race
mortality_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2414
  White Non-Hispanic                            2311
Black (includes Hispanic)                       1012
Hispanic (any race)                              572
  White Hispanic                                 560
Asian / Pacific Islander (includes Hispanic)     286
Amer. Indian / AK Native (includes Hispanic)     186
Name: race_ethnicity, dtype: int64

In [6]:
#clean race values
mortality_df['race_ethnicity'] = mortality_df['race_ethnicity'].str.strip()

In [7]:
#get count of values for race
mortality_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2414
White Non-Hispanic                              2311
Black (includes Hispanic)                       1012
Hispanic (any race)                              572
White Hispanic                                   560
Asian / Pacific Islander (includes Hispanic)     286
Amer. Indian / AK Native (includes Hispanic)     186
Name: race_ethnicity, dtype: int64

In [8]:
#drop rows where race equals white non-hispanic and race equals white hispanic to avoid overlap
mortality_df = mortality_df[mortality_df["race_ethnicity"] != "White Non-Hispanic"]
mortality_df = mortality_df[mortality_df["race_ethnicity"] != "White Hispanic"]

In [9]:
#get count of values for race
mortality_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2414
Black (includes Hispanic)                       1012
Hispanic (any race)                              572
Asian / Pacific Islander (includes Hispanic)     286
Amer. Indian / AK Native (includes Hispanic)     186
Name: race_ethnicity, dtype: int64

In [10]:
#create column for 5-year average median income 
mortality_df["5_year_avg_income"] = (mortality_df["median_household_income_2015"]+mortality_df["median_household_income_2016"]+mortality_df["median_household_income_2017"]+mortality_df["median_household_income_2018"]+mortality_df["median_household_income_2019"])/5
mortality_df.head()

,average_annual_count,recent_trend,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,five_year_trend_rates,county_state,...,popestimate2016,popestimate2017,popestimate2018,popestimate2019,median_household_income_2015,median_household_income_2016,median_household_income_2017,median_household_income_2018,median_household_income_2019,5_year_avg_income
0,181,stable,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,213.2,213.2,Walker County AL,...,64533,63895,63669,63521,37305.0,39511.0,41925.0,45434.0,45991.0,42033.2
2,10,stable,Alabama,All Cancer Sites,Black (includes Hispanic),All Sexes,All Ages,281.1,281.1,Walker County AL,...,64533,63895,63669,63521,37305.0,39511.0,41925.0,45434.0,45991.0,42033.2
3,32,stable,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,211.2,211.2,Crenshaw County AL,...,13916,13858,13825,13772,36759.0,37374.0,38419.0,39057.0,43309.0,38983.6
5,7,stable,Alabama,All Cancer Sites,Black (includes Hispanic),All Sexes,All Ages,175.2,175.2,Crenshaw County AL,...,13916,13858,13825,13772,36759.0,37374.0,38419.0,39057.0,43309.0,38983.6
6,78,falling,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,201.7,201.7,Russell County AL,...,58185,56985,57784,57961,36124.0,38617.0,36186.0,39104.0,43670.0,38740.2


In [12]:
#drop rows where avg income is null
mortality_df = mortality_df[mortality_df["5_year_avg_income"].isna() == False]

In [13]:
#get count of rows
mortality_df.count()

average_annual_count                   4463
recent_trend                           4463
primary_state_name                     4463
cancer                                 4463
race_ethnicity                         4463
sex                                    4463
age                                    4463
age_adjusted_rate                      4463
five_year_trend_rates                  4463
county_state                           4463
index_of_medical_underservice_score    3962
popestimate2015                        4463
popestimate2016                        4463
popestimate2017                        4463
popestimate2018                        4463
popestimate2019                        4463
median_household_income_2015           4463
median_household_income_2016           4463
median_household_income_2017           4463
median_household_income_2018           4463
median_household_income_2019           4463
5_year_avg_income                      4463
dtype: int64

In [14]:
#create target 
y = mortality_df[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
0,213.2
2,281.1
3,211.2
5,175.2
6,201.7


In [15]:
#create feautures variable holding ages and income
X = mortality_df[["race_ethnicity", "5_year_avg_income"]]
X.head()

,race_ethnicity,5_year_avg_income
0,White (includes Hispanic),42033.2
2,Black (includes Hispanic),42033.2
3,White (includes Hispanic),38983.6
5,Black (includes Hispanic),38983.6
6,White (includes Hispanic),38740.2


In [16]:
#create dummy variables for race
X = pd.get_dummies(X, columns=["race_ethnicity"])
X.head()

,5_year_avg_income,race_ethnicity_Amer. Indian / AK Native (includes Hispanic),race_ethnicity_Asian / Pacific Islander (includes Hispanic),race_ethnicity_Black (includes Hispanic),race_ethnicity_Hispanic (any race),race_ethnicity_White (includes Hispanic)
0,42033.2,0,0,0,0,1
2,42033.2,0,0,1,0,0
3,38983.6,0,0,0,0,1
5,38983.6,0,0,1,0,0
6,38740.2,0,0,0,0,1


In [17]:
#get shape of target array
y.shape

(4463, 1)

In [18]:
#get shape of feautures array
X.shape

(4463, 6)

In [19]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
#create linear regression model using sklearn
model = LinearRegression()

In [21]:
#fit the model with training data
regression_race_inc_mortality = model.fit(X_train, y_train)

In [22]:
#Create predictions array using linear regression
y_pred = regression_race_inc_mortality.predict(X_test)
y_pred

array([[185.58018681],
       [ 89.21252181],
       [165.58331991],
       ...,
       [158.01460378],
       [167.11977567],
       [176.2058012 ]])

In [23]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.4843753483955803

In [24]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [25]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [26]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     735.2
Date:                Thu, 02 Jun 2022   Prob (F-statistic):               0.00
Time:                        18:25:47   Log-Likelihood:                -21937.
No. Observations:                4463   AIC:                         4.389e+04
Df Residuals:                    4457   BIC:                         4.392e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [27]:
#fit the model with training data
#decision tree regression
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [28]:
#predict
y_pred = regressor_DT.predict(X_test)
y_pred

array([198.6,  68.3, 181.6, ..., 113.3, 174.4, 162.6])

In [29]:
#calculate r2 score
r2_score(y_test, y_pred)

-0.14304790198314454